In [1]:
import pandas as pd
import datetime
s=pd.Timestamp.today().normalize()  

In [2]:
today=s.strftime("%Y-%m-%d")
print(today)
days_28_period= s - datetime.timedelta(29)
days_28_period=days_28_period.strftime("%Y-%m-%d")
days_28_period

2023-03-18


'2023-02-17'

In [4]:
from pandas import json_normalize
import json
import matplotlib.pyplot as plt
#import requests module
import requests
from requests.auth import HTTPBasicAuth
page=1
sales_data=[]
while(1):
    response = requests.get('https://api.cin7.com/api/v1/SalesOrders?rows=250&where= createdDate<'+str(today)+'T00:00:00Z and createdDate>'+days_28_period+'T23:59:59Z&fields=createdDate,lineItems&page='+str(page),auth = HTTPBasicAuth('DiggsIncUS', '9901f725991a42c4b8ea12917e3c21d9'))
    if not response.json():
        break
    #print (response.json())
    page_data=response.json()
    #separarting each object from the the output array and then appending it sales_Data to have an array of salesrecord objects
    sales_data += page_data
    #end of a single page
    #print("-------------------------------------------------") 
    #increasing the page value to iterate through all pages and get the data
    page+=1 
#print(sales_data)                    


In [7]:
sales_data

[{'createdDate': '2023-03-17T23:44:55Z',
  'lineItems': [{'id': 192441,
    'createdDate': '2023-03-17T23:49:03Z',
    'transactionId': 85727,
    'parentId': 0,
    'productId': 188,
    'productOptionId': 135,
    'integrationRef': '0',
    'sort': 10,
    'code': 'GRO006AASSR',
    'name': 'Groov - Sage',
    'option1': 'Sage',
    'option2': 'OS',
    'option3': '',
    'qty': 1.0,
    'styleCode': 'GRO006AASSR',
    'barcode': '842469000753',
    'sizeCodes': None,
    'lineComments': None,
    'unitCost': 4.9253,
    'unitPrice': 34.0,
    'discount': 0.0,
    'qtyShipped': 0.0,
    'holdingQty': 0.0,
    'accountCode': '',
    'stockControl': 'FIFO'},
   {'id': 192442,
    'createdDate': '2023-03-17T23:49:03Z',
    'transactionId': 85727,
    'parentId': 0,
    'productId': 467,
    'productOptionId': 428,
    'integrationRef': '0',
    'sort': 10,
    'code': 'STP1V0210OZPBAB',
    'name': 'Treat Spread - Gold N Delicious',
    'option1': "Gold'n Delicious",
    'option2': '10 

In [8]:
df_sales = json_normalize(sales_data)
df=df_sales.explode('lineItems').reset_index(drop=True)
df=df.dropna().reset_index(drop=True)
df['code']=''
df['qty']=''
for i in range(len(df)):
    df.loc[i, "code"]=df.loc[i, "lineItems"]['code']
    df.loc[i, "qty"]=int(df.loc[i, "lineItems"]['qty'])
    
df=df.filter(['code','qty'], axis=1)
df

,code,qty
0,GRO006AASSR,1
1,STP1V0210OZPBAB,1
2,STP3V0210OZPBGM,1
3,STP1V0210OZPBAB,1
4,SC0S0001V04G,1
...,...,...
8862,PAD020AAMG,1
8863,PAD021AASC,1
8864,CBA003STDA,2
8865,FWB003ASMA,2


In [10]:
#accessing the skumapping excel
compare_df=pd.read_excel('/Users/nidhimenon/Downloads/DiggsSKUMapping.xlsx')
compare_df=compare_df.filter(['Diggs SKU', 'Variant'])
compare_df=compare_df.rename(columns={'Diggs SKU':'code'})
#compare_df
#matching the data from sales to the skumapping excel sheet to get product variants and their quantity sold
final_df=pd.merge(df, compare_df, on='code', how='left')
#final_df.to_excel("/Users/nidhimenon/Desktop/output_nan.xlsx")
final_df=final_df.groupby("Variant")["qty"].sum()
final_df=final_df.sort_values(ascending=False)
#print(final_df)

In [11]:
final_df

Variant
Groov - Turquoise                      860
Spreadable Treat - Gold N Delicious    829
Groov - Blush                          579
Spreadable Treat - Thats My Jam        568
Crate Bowl Attachment Charcoal         466
                                      ... 
Collar - Large - Slate                   1
Dispenser / Poop Bag Bundle - Lilac      1
Collar - Large - Navy                    1
V1 Snooz - Medium - Light Grey           1
Collar - Large - Blush                   1
Name: qty, Length: 135, dtype: int64